# Métodos Numéricos em Python - Aula 2

## Sumário

- [Introdução, importando bibliotecas e definindo parâmetros](#Introdução);
- [Mais sobre estruturas de dados](#Mais-sobre-estruturas-de-dados);
- [Classes e a Programação orientada ao Objeto](#Classes-e-a-Programação-orientada-ao-Objeto);
- [Exemplos resolvidos](#Exemplos-resolvidos);

## Introdução

O normal do nosso fluxo de trabalho é iniciar importando as bibliotecas que vamos utilizar no decorrer do material.
Um maior detalhamento sobre elas já foi feito na aula anterior, de modo que agora podemos utilizar diretamente:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.integrate
from tqdm.notebook import tqdm

O bloco a seguir é opcional, ele vai alterar o estilo padrão de nossas figuras, e aumentar um pouco o seu tamanho, melhorando a apresentação em nossa aula:

In [ ]:
# Definindo um novo estilo para as figuras [opcional]
# Esse bloco modifica alguns dos valores padrões para

plt.rcdefaults()

# https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
plt.style.use("ggplot")

# https://matplotlib.org/3.1.1/tutorials/introductory/customizing.html
plt.rcParams.update({"figure.dpi": 100, "figure.figsize": (6, 6)})

## Mais sobre estruturas de dados

Vamos revisitar o tópico estruturas de dados, dada a sua importância para qualquer linguagem de programação. Conhecer as suas peculiaridades nos permite escrever códigos mais concisos e legíveis, tornando-os menos propensos a erros e mais fáceis de aprimorar, colaborar e compartilhar.

### Motivação

Vamos estabelecer um exemplo prático, duas coordenadas espaciais (`x` e `y`) e uma temporal (`t`), para exemplificar o que realmente faríamos em aplicações de métodos numéricos e turbulência:

In [ ]:
x = np.linspace(start = 0.0, stop = 10.0, num = 5)
y = np.linspace(start = 0.0, stop = 20.0, num = 5)
t = np.linspace(start = 0.0, stop = 60.0, num = 5)

Podemos acessar cada uma delas pelos endereços `x`, `y` e `t`, tente no bloco abaixo:

In [ ]:
print(x)

Mais isso acarreta em dois possíveis problemas práticos:

- O nome das variáveis não expressa significativamente seu conteúdo (mais propensos a erros e mais difícil de aprimorar, colaborar e compartilhar);
- Variáveis de apenas um caracter podem ser descuidadamente sobrescritas em outros trechos de código de nossa aplicação.

Vamos ver este exemplo de **Fortran vs Python** para exemplificar:

```fortran
program Teste

    implicit none

    real, dimension(5) :: x

    x = 0.0 ! Todas as componentes do vetor x serão zero,
            ! mas pela tipagem estática, x continuará sendo um vetor

end program Teste
```

**Atenção**: Python possui tipagem dinâmica, então o seguinte bloco tem um comportamento diferenciado:

In [ ]:
x = 0.0

A variáveis `x` era um endereço pelo qual acessávamos um vetor, de modo que perdemos o acesso a ele, pois `x` agora endereça outro objeto (o número `0.0`).

In [ ]:
print(x)

É considerada uma boa prática, dar nome mais significativo para as variáveis:

In [ ]:
coord_x = np.linspace(start = 0.0, stop = 10.0, num = 5)
coord_y = np.linspace(start = 0.0, stop = 20.0, num = 5)
coord_t = np.linspace(start = 0.0, stop = 60.0, num = 5)

São três variáveis para ficarmos lembrando, verificando e testando. Isso pode parecer banal, mas não se engane, as coordenadas são apenas a ponta o iceberg para os problemas que queremos resolver.

Mas perceba que as três tem uma função muito semelhante, apenas com alguns parâmetros levemente diferentes, podemos melhorar isso?

### Dicionários

Dicionário são estruturas de dados caracterizados pelos conjuntos de pares `key: value`, com a exigência de que as chaves sejam **exclusivas** (dentro de um dicionário).

As operações principais em um dicionário estão armazenando um valor com alguma chave e extraindo o valor dado a chave.

Veja duas formas equivalentes de iniciá-los:

In [ ]:
coord = dict(
    x = np.linspace(start = 0.0, stop = 10.0, num = 5),
    y = np.linspace(start = 0.0, stop = 20.0, num = 5),
    t = np.linspace(start = 0.0, stop = 60.0, num = 5)
)

coord = {
    "x": np.linspace(start = 0.0, stop = 10.0, num = 5),
    "y": np.linspace(start = 0.0, stop = 20.0, num = 5),
    "t": np.linspace(start = 0.0, stop = 60.0, num = 5)
}

Podemos acessar cada entrada por meio de suas chaves, isso é:

* `coord["x"]`;
* `coord["y"]`;
* `coord["t"]`.

E também temos a facilidade de poder verificar todas as mesmo tempo:

In [ ]:
coord

Podemos adicionar novos valores:

In [ ]:
coord["z"] = np.linspace(start = 0.0, stop = 10.0, num = 5)

Remover valores:

In [ ]:
del coord["t"]

E modificar os valores já existentes:

In [ ]:
coord["x"] = 0.0

Veja o resultado final:

In [ ]:
coord

Veja mais sobre dicionários na [Documentação Python](https://docs.python.org/pt-br/3/tutorial/datastructures.html#dictionaries).

### Tuplas

Tuplas (do inglês *tuple*) são outro tipo útil para armazenar dados. Tem um comportamento similar à listas, mas com uma diferença fundamental: **Tuplas são objetos imutáveis**.

Sua construção é indicada pelos parênteses `( )`, ou pelo contrutor equivalente `tuple()`, veja o exemplo:

In [ ]:
coord = (
    np.linspace(start = 0.0, stop = 10.0, num = 5),
    np.linspace(start = 0.0, stop = 20.0, num = 5),
    np.linspace(start = 0.0, stop = 60.0, num = 5)
)

Assim como listas, acessamos as informações pelo número inteiro correspondente a posição:

In [ ]:
coord[0]

In [ ]:
coord[2]

In [ ]:
coord

Note que o acesso por meio da numeração pode não ser a maneira mais legível para a sua aplicação, assim, Python oferece uma estrutura especializada denominada [namedtuple](https://docs.python.org/pt-br/3/library/collections.html#collections.namedtuple). Assim como outras estruturas, ela acompanha a instalação Python, mas ainda assim, precisamos importa-las junto com o pacote collections:

In [ ]:
import collections

Criamos uma descrição das variáveis que deremos armazenar de maneira nomeada destro desta tupla especial, com o nome do conjunto, e suas propriedades:

In [ ]:
Coordenada = collections.namedtuple("Coordenadas", ["x", "y", "t"])

In [ ]:
coord = Coordenada(
    x = np.linspace(start = 0.0, stop = 10.0, num = 5),
    y = np.linspace(start = 0.0, stop = 20.0, num = 5),
    t = np.linspace(start = 0.0, stop = 60.0, num = 5)
)

Podemos agora acessas as informações tanto por meio do numeração inteira quanto por meio de seus nomes:

In [ ]:
coord[0]

In [ ]:
coord.x * 10

## Classes e a Programação orientada ao Objeto

Classes são um recurso importante da linguagem para definir propriedades e métodos personalizados para um dado objeto.

Embora de fato podemos resolver diversos problemas em Python sem precisar recorrer à programação de classes, compreender o básico de seu funcionamento é muito importante, pois muito do universo da linguagem e também dos pacotes especializados que utilizamos se aproveitam dos benefícios da programaão orientada do objeto.

In [ ]:
inteiro = 1

In [ ]:
type(1)

Mesmo um simples número inteiro tem uma série de atributos e métodos, que muitas vezes nem nos damos conta:

In [ ]:
dir(int)

Para exempliciar, vamos continuar utilizando nosso exemplo anterior, e tratar das coordendas que precisamos para resolver um dado problema.

Sistema de coordenadas possuem uma série de propriedades, por exemplo:

* O próprio vetor posição;
* Um espaçamento (ou delta), vamos nos mantes nos casos de espaçamento uniforme por simplicidade;
* Um nome que a descreva;
* E a representação matemática.

Uma classe, em Python, é a maneira como descrevemos os objetos. A primeira versão da nossa descrição de uma coordenada pode ser feita da seguinte maneira:

In [ ]:
class Coordenada_v0:

    def __init__(self, inicio, final, numero_de_pontos, nome, simbolo):
        """Esse método é invocado durante a inicialização de uma instância da classe"""
        
        self.vetor = np.linspace(
            start = inicio,
            stop = final,
            num = numero_de_pontos
        )
        self.delta = (final - inicio) / (numero_de_pontos - 1.0)
        self.nome = nome
        self.simbolo = simbolo

E uma vez que a classe esteja criada, podemos instanciar um objeto do tipo Coordenada com o seguinte bloco:

In [ ]:
coord_x = Coordenada_v0(
    inicio = 0.0,
    final = 10.0,
    numero_de_pontos = 21,
    nome = "x",
    simbolo = r"$x$"    
)

E acessamos todas as suas propriedades por meio da notação `.` (ponto), veja só:

In [ ]:
coord_x.vetor

In [ ]:
coord_x.nome

In [ ]:
coord_x.delta

In [ ]:
coord_x.simbolo

In [ ]:
coord_x # Veremos como personalizar a exibição na tela a seguir

Também podemos atribur mais atributos posteriormente, ou mesmo modificar os atributos já existentes:

In [ ]:
coord_x.nova_propriedade = None

In [ ]:
coord_x.vetor = "qualquer outra coisa"

In [ ]:
coord_x.vetor

Mas note que isso pode resultar em um comportamente inesperado, nosso vetor é agora uma frase!

O que podemos fazer quanto à isso é "proteger" alguns dos atributos ao iniciar seus nomes com um underline `_`, e tornar seu acesso possível por meio de propriedades. Também, vamos adicionar um método `__repr__` personalizado, ele é responvável por exibir informações do nosso objeto na tela, quando demandado.

Veja como ficou o código:

In [ ]:
class Coordenada_v1:

    def __init__(self, inicio, final, numero_de_pontos, nome, simbolo):
        
        self._vetor = np.linspace(inicio, final, numero_de_pontos)
        self._delta = (final - inicio) / (numero_de_pontos - 1.0)
        self._nome = nome
        self.simbolo = simbolo
    
    def __repr__(self):
        return f"Coordenada({self.inicio}, {self.final}, {self.numero_de_pontos}, {self.nome}, {self.simbolo})"

    @property
    def vetor(self):
        return self._vetor
    @property
    def delta(self):
        return self._delta
    @property
    def inicio(self):
        return self._vetor[0]
    @property
    def final(self):
        return self._vetor[-1]
    @property
    def comprimento(self):
        return self._vetor[-1] - self._vetor[0]
    @property
    def nome(self):
        return self._nome
    @property
    def numero_de_pontos(self):
        return self._vetor.size
    @property
    def size(self):
        return self.numero_de_pontos

In [ ]:
coord_x = Coordenada_v1(
    inicio = 0.0,
    final = 10.0,
    numero_de_pontos = 11,
    nome = "x",
    simbolo = r"$x$"    
)

In [ ]:
coord_x # Melhor, não?

Perceba que agora atribuir outros valores ao vetor irá resultar em um erro, tente fazer isso:

In [ ]:
coord_x.vetor

Mas a não ser em situações muito específicas, nós não começamos uma classe do zero, como vimos no exemplo anterior. Nós usamos a propriedade de herança para personalizar classes já existentes, de modo que elas se adequem ao funcionamento que esperamos para nossa aplicação. Isso é feito pela linha inicial `class Coordenada_v2(Coordenada_v1)`. A classe `Coordenada_v2` será capaz de fazer tudo o que já continha em `Coordenada_v1`, além de novos métodos para ser capaz de lidar com operadores de soma `+` e subtração `-`. Veja o código:

In [ ]:
class Coordenada_v2(Coordenada_v1):

    def __add__(self, other):
        return self._vetor + other
    
    def __radd__(self, other):
        return other + self._vetor
    
    def __sub__(self, other):
        return self._vetor - other
    
    def __rsub__(self, other):
        return other - self._vetor
   
    def __len__(self):
        return self.vetor.size

In [ ]:
coord_x = Coordenada_v2(
    inicio = 0.0,
    final = 10.0,
    numero_de_pontos = 11,
    nome = "x",
    simbolo = r"$x$"    
)

In [ ]:
coord_x

In [ ]:
coord_x + 4

In [ ]:
coord_x - 8

E assim podemos ir construindo nossa classe. Existe uma vasta quantidade de métodos especiais que podem ser atribuidos, veja a [documentação](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types) para mais detalhes. Além disso, podemos implementar nossos próprios métodos, e ampliar o poder das classes.

Mas como vimos anteriormente, nem sempre precisamos implementar tudo à mão, podemos tirar proveito de objetos e classes fornecidos por Python ou por suas bibliotecas.

Nossas coordenadas podem se derivar dos próprios vetores em Numpy (classe `np.ndarray`) por herança, e trazerem consigo uma vasta gama de propriedades e métodos numéticos. Para mais detalhes, consulte o artigo [Subclassing ndarray](https://numpy.org/doc/stable/user/basics.subclassing.html).

Veja o código:

In [ ]:
class Coordenada(np.ndarray):

    def __new__(cls, inicio, final, numero_de_pontos, nome, simbolo):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(
            np.linspace(inicio, final, numero_de_pontos, dtype=np.float64)
        ).view(cls)
        # add the new attribute to the created instance
        obj._delta = (final - inicio) / (numero_de_pontos - 1.0)
        obj._nome = nome
        obj.simbolo = simbolo
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # see InfoArray.__array_finalize__ for comments
        if obj is None: return
        self._delta = getattr(obj, '_delta', None)
        self._nome = getattr(obj, '_nome', "x")
        self.simbolo = getattr(obj, 'simbolo', r"$x$")

    def __repr__(self):
        return f"Coordenada({self.inicio}, {self.final}, {self.size}, {self.nome}, {self.simbolo})"
        
    @property
    def delta(self):
        return self._delta
    @property
    def inicio(self):
        return self[0]
    @property
    def final(self):
        return self[-1]
    @property
    def comprimento(self):
        return self.final - self.inicio
    @property
    def nome(self):
        return self._nome

In [ ]:
coord_x = Coordenada(
    inicio = 0.0,
    final = 10.0,
    numero_de_pontos = 101,
    nome = "x",
    simbolo = r"$x$"    
)

In [ ]:
coord_x * 10

In [ ]:
coord_x ** 20

In [ ]:
np.sin(coord_x)

Agora que temos um coordenada funcional, camos criar uma classe especializada para os problemas bidimenionais que iremos resolver. Note que ela aceita os parametros e armazena as coordendas `x` e `y`:

In [ ]:
class Coordenadas2d:
    
    def __init__(self, parametros_x: dict, parametros_y: dict):
        
        self._x = Coordenada(**parametros_x)
        self._y = Coordenada(**parametros_y)
        
    @property
    def x(self): return self._x
    @property
    def y(self): return self._y

In [ ]:
coord = Coordenadas2d(
    parametros_x = dict(
        inicio = 0.0,
        final = 10.0,
        numero_de_pontos = 101,
        nome = "componente horizontal",
        simbolo = r"$x$"
    ),
    parametros_y = dict(
        inicio = 0.0,
        final = 10.0,
        numero_de_pontos = 101,
        nome = "componente vertical",
        simbolo = r"$y$"
    ),
)

Acessamos os atributos individualmente:

In [ ]:
coord.x

In [ ]:
coord.x.delta

In [ ]:
coord.y + 20.

De maneira semelhante, podemos descrever uma dada matriz ou campo bidimensional como:

* Um arranjo numérico;
* Um nome próprio;
* Uma representação matemática.

Veja o código:

In [ ]:
class Campo2d_v0(np.ndarray):

    def __new__(cls, coord: Coordenadas2d, nome: str, simbolo: str):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(
            np.zeros(shape=(coord.x.size, coord.y.size), dtype=np.float64)
        ).view(cls)
        # add the new attribute to the created instance
        obj._coord = coord
        obj._nome = nome
        obj._simbolo = simbolo
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # see InfoArray.__array_finalize__ for comments
        if obj is None: return
        self._coord = getattr(obj, '_coord', None)
        self._nome = getattr(obj, '_nome', "x")
        self._simbolo = getattr(obj, '_simbolo', r"$x$")

    @property
    def coord(self): return self._coord
    @property
    def nome(self): return self._nome
    @property
    def simbolo(self): return self._simbolo

In [ ]:
ux = Campo2d_v0(coord, "velocidade horizontal", r"$u_x$")
ux

Vamos adicionar métodos personalizados para imprimir a arranjo na tela, caso necessário. E também, uma vez que as coordenadas estão disponíveis como atributos, podemos criar um método para graficar a matriz na tela:

In [ ]:
class Campo2d_v1(Campo2d_v0):

    def __str__(self):
        return f"{self.nome} - {self.simbolo}: mean={self.mean()}, min={self.min()}, max={self.max()}"
        
    def plot(self, figure_name = None):
        fig, ax = plt.subplots()
        
        im = ax.contourf(self.coord.x, self.coord.y, self.T)
        fig.colorbar(im)
        
        ax.set_xlabel(f"{self.coord.x.nome} - {self.coord.x.simbolo}")
        ax.set_ylabel(f"{self.coord.y.nome} - {self.coord.y.simbolo}")
        ax.set_title(f"{self.nome} - {self.simbolo}")
        ax.set_aspect('equal', 'box')
        
        if figure_name is None:
            fig.show()
        else:
            fig.savefig(figure_name)
            plt.close(fig)
        

In [ ]:
ux = Campo2d_v1(coord, "velocidade horizontal", r"$u_x$")

In [ ]:
print(ux)

In [ ]:
ux.plot()

Lembre-se que na aula passada implementamos métodos numérios para obtenção da derivada primeira e segunda:

In [ ]:
def derivada_primeira(array, delta):
    
    derivada = np.zeros_like(array)
    
    derivada[0] = - 3.0 * array[0] + 4.0 * array[1] - 1.0 * array[2]
    derivada[1:-1] = array[:-2] - array[2:]
    derivada[-1] = 1.0 * array[-3] - 4.0 * array[-2] + 3.0 * array[-1]
    
    return derivada / (2.0 * delta)

def derivada_segunda(array, delta):
    derivada = np.zeros_like(array)
    
    derivada[0] = 2.0 * array[0] - 5.0 * array[1] + 4.0 * array[2] - 1.0 * array[3]
    derivada[1:-1] = array[2:] - 2.0 * array[1:-1] + array[0:-2]
    derivada[-1] = - array[-4] + 4.0 * array[-3] - 5.0 * array[-2] + 2.0 * array[-1]
    
    return derivada / delta ** 2.0

Vamos acrescentar essas operações para serem acessadas facilmente em nossos arranjos:

In [ ]:
class Campo2d_v2(Campo2d_v1):

    @property
    def dx(self):
        return np.apply_along_axis(derivada_primeira, 0, self, self.coord.x.delta)

    @property
    def dxx(self):
        return np.apply_along_axis(derivada_segunda, 0, self, self.coord.x.delta)
    
    @property
    def dy(self):
        return np.apply_along_axis(derivada_primeira, 1, self, self.coord.y.delta)

    @property
    def dyy(self):
        return np.apply_along_axis(derivada_segunda, 1, self, self.coord.y.delta)

In [ ]:
ux = Campo2d_v2(coord, "velocidade horizontal", r"$u_x$")

Podemos agora inclusive realizar os diversos métodos em cadeia, e aqui, começamos a realmente mostra a profunda utilidade das classes:

In [ ]:
ux.dx.plot()

Por fim, vamos adicionar ao nosso exemplo métodos práticos para a resolução de problemas em fenômenos de transporte, sendo eles:

* Calcular o termo convectivo/difusivo de uma função $f$, dada às velocidades $u_x$, $u_y$ e o coeficiente de difusão:

    $$
    - u_x \dfrac{\partial f}{\partial x} - u_y \dfrac{\partial f}{\partial y} + \dfrac{1}{diff_{coef}} \left( \dfrac{\partial^2 f}{\partial x^2} + \dfrac{\partial^2 f}{\partial y^2} \right)
    $$

* Resolver a equação de pressão-Poisson discreta, que pode ser escrita como 

    $$
    \begin{split}
    p_{i,j}^{n} = & \frac{\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}\right) \Delta y^2 + \left(p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) \Delta x^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
    & -\frac{\Delta x^2\Delta y^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
    & \times \left[\frac{1}{\Delta t}\left(\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}+\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right)-\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x} -2\frac{u_{i,j+1}-u_{i,j-1}}{2\Delta y}\frac{v_{i+1,j}-v_{i-1,j}}{2\Delta x}-\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right]
    \end{split}
    $$
    
    Mais detalhes podem ser obtidos nos passos 10 e 11 do curso [CFD com Python](https://github.com/fschuch/CFDPython-BR).

In [ ]:
class Campo2d_v3(Campo2d_v2):

    def convectivo_difusivo(self, ux, uy, diff_coef):
        return - ux * self.dx - uy * self.dy + (self.dxx + self.dyy) / diff_coef
    
    def resolver_poisson(self, ux, uy, dt, numero_iterações=50):
        array_b = (ux.dx + uy.dy)*0.5/dt - (ux.dx*0.5)**2.0 - uy.dx * ux.dy - (uy.dy*0.5)**2.0
        
        for n in range(numero_iterações):
            pn = self.copy()
            self[1:-1, 1:-1] = (((pn[2:,1:-1] + pn[0:-2,1:-1]) * self.coord.y.delta**2 + 
                          (pn[1:-1,2:] + pn[1:-1,0:-2]) * self.coord.x.delta**2) /
                          (2 * (self.coord.x.delta**2 + self.coord.y.delta**2)) -
                          self.coord.x.delta**2 * self.coord.y.delta**2 / (2 * (self.coord.x.delta**2 + self.coord.y.delta**2)) * 
                          array_b[1:-1,1:-1])
            
            self[-1,:] = self[-2,:] # dp/dx = 0 at x = Lx
            self[:,0] = self[:,1]   # dp/dy = 0 at y = 0
            self[0,:] = self[1,:]   # dp/dx = 0 at x = 0
            self[:,-1] = self[:,-2] # dp/dy = 0 at y = Ly        

## Exemplos resolvidos

### Corrente de Densidade em Lançamento Finito

<img src="../assets/output.gif">

$$\frac{\partial^2 p}{\partial x^2}+\frac{\partial^2 p}{\partial y^2} = -\left(\frac{\partial u}{\partial x}\frac{\partial u}{\partial x}+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\frac{\partial v}{\partial y}\frac{\partial v}{\partial y} \right)$$

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = - \frac{\partial p}{\partial x}+ \frac{1}{Re} \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right) $$

$$\frac{\partial v}{\partial t}+u\frac{\partial v}{\partial x}+v\frac{\partial v}{\partial y} = - \frac{\partial p}{\partial y}+ \frac{1}{Re} \left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2}\right) - c $$

$$ \frac{\partial c}{\partial t} + u\frac{\partial c}{\partial x} + v\frac{\partial c}{\partial y} = \frac{1}{Re ~ Sc} \left(\frac{\partial^2 c}{\partial x^2}+\frac{\partial^2 c}{\partial y^2} \right) $$

A equação do momento na direção de $u$:

$$
\begin{split}
u_{i,j}^{n+1} = u_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(u_{i,j}^{n}-u_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(u_{i,j}^{n}-u_{i,j-1}^{n}\right) \\
& - \frac{\Delta t}{2\Delta x} \left(p_{i+1,j}^{n}-p_{i-1,j}^{n}\right) \\
& + \frac{1}{Re} \left(\frac{\Delta t}{\Delta x^2} \left(u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}\right)\right)
\end{split}
$$

A equação do momento na direção de $v$:

$$
\begin{split}
v_{i,j}^{n+1} = v_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(v_{i,j}^{n}-v_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(v_{i,j}^{n}-v_{i,j-1}^{n})\right) \\
& - \frac{\Delta t}{2\Delta y} \left(p_{i,j+1}^{n}-p_{i,j-1}^{n}\right) \\
& + \frac{1}{Re} \left(\frac{\Delta t}{\Delta x^2} \left(v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}\right)\right) - \Delta t c_{i,j}
\end{split}
$$

Quase lá! Agora, reorganizamos a equação de pressão-Poisson:

$$
\begin{split}
p_{i,j}^{n} = & \frac{\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}\right) \Delta y^2 + \left(p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) \Delta x^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
& -\frac{\Delta x^2\Delta y^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
& \times \left[\frac{1}{\Delta t}\left(\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}+\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right)-\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x} -2\frac{u_{i,j+1}-u_{i,j-1}}{2\Delta y}\frac{v_{i+1,j}-v_{i-1,j}}{2\Delta x}-\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right]
\end{split}
$$

E a equação do transporte de escalar:

$$
\begin{split}
c_{i,j}^{n+1} = c_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(c_{i,j}^{n}-c_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(c_{i,j}^{n}-c_{i,j-1}^{n})\right) \\
& + \frac{1}{Re ~Sc} \left(\frac{\Delta t}{\Delta x^2} \left(c_{i+1,j}^{n}-2c_{i,j}^{n}+c_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(c_{i,j+1}^{n}-2c_{i,j}^{n}+c_{i,j-1}^{n}\right)\right)
\end{split}
$$

A condição inicial é $u, v, p = 0 $ em todos os lugares, e as condições de contorno são:

$u=1$ em $y=2$ (a "tampa");

$u, v=0$ nas fronteiras restantes;

$\frac{\partial p}{\partial y}=0$ em $y=0$;

$p=0$ em $y=2$

$\frac{\partial p}{\partial x}=0$ em $x=0$ e $x=2$

Podemos finalmente escrever o solver para Navier-Stokes, tirando proveito de todos os métodos que implementamos na nossa classe `Campo2d`. Veja o código:

In [ ]:
def lock_exchange(coordenadas, total_steps, time_step, time_snap, numero_de_reynolds, numero_de_schmidt, richardson_number):
    
    ux = Campo2d_v3(coordenadas, "velocidade horizontal", r"$u_x$")
    uy = Campo2d_v3(coordenadas, "velocidade vertical", r"$u_y$")
    p = Campo2d_v3(coordenadas, "pressão", r"$p$")
    
    c = Campo2d_v3(coordenadas, "Concentração adimensional", r"$c$")
    # Condição inicial
    c[:coordenadas.x.size//4,:] += 1.0
    
    for n in tqdm(range(total_steps)):
        uxn = ux.copy()
        uyn = uy.copy()
        cn = c.copy()
        
        p.resolver_poisson(uxn, uyn, dt=time_step)        
        ux += time_step * (uxn.convectivo_difusivo(uxn, uyn, diff_coef=numero_de_reynolds) - p.dx)
        uy += time_step * (uyn.convectivo_difusivo(uxn, uyn, diff_coef=numero_de_reynolds) - p.dy + richardson_number * cn)
        c += time_step * cn.convectivo_difusivo(uxn, uyn, diff_coef=numero_de_reynolds*numero_de_schmidt)
        
        # Condições de contorno de não deslizamento para velocidade
        ux[:,0]  = 0
        ux[0,:]  = 0
        ux[-1,:] = 0
        ux[:,-1] = 0
        uy[:,0]  = 0
        uy[:,-1] = 0
        uy[0,:]  = 0
        uy[-1,:] = 0
        
        # Fluxo nulo para concentração
        c[-1,:] = c[-2,:] # dc/dx = 0 at x = 4
        c[:,0] = c[:,1]   # dc/dy = 0 at y = 0
        c[0,:] = c[1,:]   # dc/dx = 0 at x = 0
        c[:,-1] = c[:,-2] # dc/dy = 0 at y = 1
        
        if n % time_snap == 0:
            c.plot(f"c-{str(n//time_snap).zfill(4)}")
    
    return ux, uy, p, c

Agora basta definirmos os parâmetros e invocar o solver:

In [ ]:
coordenadas = Coordenadas2d(
    parametros_x = dict(
        inicio = 0.0,
        final = 4.0,
        numero_de_pontos = 81,
        nome = "componente horizontal",
        simbolo = r"$x$"
    ),
    parametros_y = dict(
        inicio = 0.0,
        final = 1.0,
        numero_de_pontos = 21,
        nome = "componente vertical",
        simbolo = r"$y$"
    ),
)

ux, uy, p, c = lock_exchange(
    coordenadas,
    total_steps=2000,
    time_step=0.001,
    time_snap=500,
    numero_de_reynolds=100.0,
    numero_de_schmidt=1.0,
    richardson_number=1.0
)

Vamos analisar os resultados:

In [ ]:
print(ux)

In [ ]:
for var in [c, ux, uy, p]:
    var.plot()

### Escoamento em Cavidade com Transferência de Calor

Aqui está o sistema de equações diferenciais: duas equações para os componentes de velocidade $u,v$, uma equação para pressão e uma para a temperatura:

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = - \frac{\partial p}{\partial x}+ \frac{1}{Re} \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right) $$

$$\frac{\partial v}{\partial t}+u\frac{\partial v}{\partial x}+v\frac{\partial v}{\partial y} = - \frac{\partial p}{\partial y}+ \frac{1}{Re} \left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2}\right) + Ri ~ \Theta $$

$$\frac{\partial^2 p}{\partial x^2}+\frac{\partial^2 p}{\partial y^2} = -\left(\frac{\partial u}{\partial x}\frac{\partial u}{\partial x}+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\frac{\partial v}{\partial y}\frac{\partial v}{\partial y} \right)$$

$$ \frac{\partial \Theta}{\partial t} + u\frac{\partial \Theta}{\partial x} + v\frac{\partial \Theta}{\partial y} = \frac{1}{Re ~ Pr} \left(\frac{\partial^2 \Theta}{\partial x^2}+\frac{\partial^2 \Theta}{\partial y^2} \right) $$

* Equações discretas:

A equação do momento na direção de $u$:

$$
\begin{split}
u_{i,j}^{n+1} = u_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(u_{i,j}^{n}-u_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(u_{i,j}^{n}-u_{i,j-1}^{n}\right) \\
& - \frac{\Delta t}{2\Delta x} \left(p_{i+1,j}^{n}-p_{i-1,j}^{n}\right) \\
& + \frac{1}{Re} \left(\frac{\Delta t}{\Delta x^2} \left(u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}\right)\right)
\end{split}
$$

A equação do momento na direção de $v$:

$$
\begin{split}
v_{i,j}^{n+1} = v_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(v_{i,j}^{n}-v_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(v_{i,j}^{n}-v_{i,j-1}^{n})\right) \\
& - \frac{\Delta t}{2\Delta y} \left(p_{i,j+1}^{n}-p_{i,j-1}^{n}\right) \\
& + \frac{1}{Re} \left(\frac{\Delta t}{\Delta x^2} \left(v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}\right)\right) - \Delta t c_{i,j}
\end{split}
$$

Quase lá! Agora, reorganizamos a equação de pressão-Poisson:

$$
\begin{split}
p_{i,j}^{n} = & \frac{\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}\right) \Delta y^2 + \left(p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) \Delta x^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
& -\frac{\Delta x^2\Delta y^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
& \times \left[\frac{1}{\Delta t}\left(\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}+\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right)-\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x} -2\frac{u_{i,j+1}-u_{i,j-1}}{2\Delta y}\frac{v_{i+1,j}-v_{i-1,j}}{2\Delta x}-\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right]
\end{split}
$$

E a equação do transporte de escalar:

$$
\begin{split}
\Theta_{i,j}^{n+1} = \Theta_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(\Theta_{i,j}^{n}-\Theta_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(\Theta_{i,j}^{n}-\Theta_{i,j-1}^{n})\right) \\
& + \frac{1}{Re ~ Pr} \left(\frac{\Delta t}{\Delta x^2} \left(\Theta_{i+1,j}^{n}-2\Theta_{i,j}^{n}+\Theta_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(\Theta_{i,j+1}^{n}-2\Theta_{i,j}^{n}+\Theta_{i,j-1}^{n}\right)\right)
\end{split}
$$

A condição inicial é $u, v, p = 0 $ em todos os lugares, e as condições de contorno são:

$u=1$ em $y=2$ (a "tampa");

$u, v=0$ nas fronteiras restantes;

$\frac{\partial p}{\partial y}=0$ em $y=0$;

$p=0$ em $y=2$

$\frac{\partial p}{\partial x}=0$ em $x=0$ e $x=2$

In [ ]:
def cavidade_termica(coordenadas, total_steps, time_step, time_snap, numero_de_reynolds, numero_de_prandtl, richardson_number):
    
    ux = Campo2d_v3(coordenadas, "velocidade horizontal", r"$u_x$")
    uy = Campo2d_v3(coordenadas, "velocidade vertical", r"$u_y$")
    p = Campo2d_v3(coordenadas, "pressão", r"$p$")
    
    theta = Campo2d_v3(coordenadas, "Temperatura adimensional", r"$\Theta$")
    #theta += 1.0
    
    for n in tqdm(range(total_steps)):
        uxn = ux.copy()
        uyn = uy.copy()
        thetan = theta.copy()
        
        p.resolver_poisson(uxn, uyn, dt=time_step)
        
        p[:,-1] *= 0.0 # p = 0 at y = 2
        
        ux += time_step * (uxn.convectivo_difusivo(uxn, uyn, diff_coef=numero_de_reynolds) - p.dx)
        uy += time_step * (uyn.convectivo_difusivo(uxn, uyn, diff_coef=numero_de_reynolds) - p.dy - richardson_number * thetan)
        theta += time_step * thetan.convectivo_difusivo(uxn, uyn, diff_coef=numero_de_reynolds*numero_de_prandtl)
        
        # Condições de contorno de não deslizamento para velocidade
        ux[:,0]  = 0
        ux[0,:]  = 0
        ux[-1,:] = 0
        ux[:,-1] = 1  # Definir velocidade na tampa da cavidade igual a 1
        uy[:,0]  = 0
        uy[:,-1] = 0
        uy[0,:]  = 0
        uy[-1,:] = 0
        
        # Fluxo nulo para concentração
        theta[-1,:] = 1
        theta[:,0] = 1
        theta[0,:] = 1
        theta[:,-1] = 0
    
    return ux, uy, p, theta

In [ ]:
coordenadas = Coordenadas2d(
    parametros_x = dict(
        inicio = 0.0,
        final = 2.0,
        numero_de_pontos = 21,
        nome = "componente horizontal",
        simbolo = r"$x$"
    ),
    parametros_y = dict(
        inicio = 0.0,
        final = 2.0,
        numero_de_pontos = 21,
        nome = "componente vertical",
        simbolo = r"$y$"
    ),
)

ux, uy, p, theta = cavidade_termica(
    coordenadas,
    total_steps=2000,
    time_step=0.001,
    time_snap=500,
    numero_de_reynolds=100.0,
    numero_de_prandtl=1.0,
    richardson_number=1.0
)

In [ ]:
fig = plt.figure(figsize=(11, 7), dpi=100)
ax = fig.add_subplot(111)
CS = ax.contourf(coordenadas.x, coordenadas.y, theta.T, alpha=0.5, cmap=plt.cm.coolwarm)
cbar = fig.colorbar(CS)
cbar.ax.set_ylabel(theta.simbolo)
ax.contour(coordenadas.x, coordenadas.y, theta.T, cmap=plt.cm.coolwarm)
ax.streamplot(coordenadas.x, coordenadas.y, ux.T, uy.T, color='k')
ax.set_aspect('equal', 'box')
ax.set_xlabel(coordenadas.x.simbolo)
ax.set_ylabel(coordenadas.y.simbolo)
ax.set_title('Escoamento em Cavidade com Transferência de Calor');